<a href="https://colab.research.google.com/github/Agoston03/Deep-Learning-42/blob/main/deepl_42_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a homework project in "Deep Learning a gyakorlatban Python és Lua alapokon".  
The team members are:

* Gyulai Gergő László
* Horváth Ágoston
* Frink Dávid

You can read more information about our chosen homework at the link below:  
https://www.kaggle.com/competitions/isic-2024-challenge

# Download and setup

Download Kaggle

In [ ]:
!pip install kaggle==1.5.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=e0f30647ad2b67990c68b8d98166a2c4e5356d5189405045c9645f32e8886a73
  Stored in directory: /root/.cache/pip/wheels/70/0c/e6/79103212a102e78b8453691b905f48000219574ba7137e7207
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.17
    Uninstalling kaggle-1.6.17:
      Successfully uninstalled kaggle-1.6.17


Configure Kaggle to access the API  
**Warning!** You need to copy your own kaggle.json file into Colab in order to valide yourself

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Download the data

In [ ]:
!kaggle competitions download -c isic-2024-challenge

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


Unpacking the data  
**Warning!** This might take a few minuttes

In [ ]:
!unzip isic-2024-challenge.zip

unzip:  cannot find or open isic-2024-challenge.zip, isic-2024-challenge.zip.zip or isic-2024-challenge.zip.ZIP.


# Preparing train, test and valid set

Gather information about the dataset based on the metadata

In [ ]:
import pandas as pd

In [ ]:
metadata = pd.read_csv('train-metadata.csv')
metadata.head()

FileNotFoundError: [Errno 2] No such file or directory: 'train-metadata.csv'

Lets check the number of benign and malignant data

In [ ]:
benign_data = metadata[metadata['target'] == 0]
malignant_data = metadata[metadata['target'] == 1]

print(f'Benign images: {len(benign_data)}')
print(f'Malignant images: {len(malignant_data)}')

In [ ]:
benign_ids = benign_data['isic_id'].tolist()
malignant_ids = malignant_data['isic_id'].tolist()

print(f'Benign images ids: {benign_ids[:5]}')
print(f'Malignant images ids: {malignant_ids[:5]}')

In [ ]:
test_split = 0.1
valid_split = 0.1

test_benign_ids = benign_ids[:int(len(benign_ids) * test_split)]
test_malignant_ids = malignant_ids[:int(len(malignant_ids) * test_split)]

valid_benign_ids = benign_ids[int(len(benign_ids) * test_split):int(len(benign_ids) * (test_split + valid_split))]
valid_malignant_ids = malignant_ids[int(len(malignant_ids) * test_split):int(len(malignant_ids) * (test_split + valid_split))]

train_benign_ids = benign_ids[int(len(benign_ids) * (test_split + valid_split)):]
train_malignant_ids = malignant_ids[int(len(malignant_ids) * (test_split + valid_split)):]

print(f'Test benign images: {len(test_benign_ids)}')
print(f'Test malignant images: {len(test_malignant_ids)}')
print(f'Valid benign images: {len(valid_benign_ids)}')
print(f'Valid malignant images: {len(valid_malignant_ids)}')
print(f'Train benign images: {len(train_benign_ids)}')
print(f'Train malignant images: {len(train_malignant_ids)}')

### Loadig the images

Prepare to load and show images

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import io
import matplotlib.image as mpimg
import h5py

Visualize some of the data we have

In [ ]:
# megadjuk, hogy hány sor és oszlop jelenjen meg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols * 3, nrows * 3)

next_benign_pix = [key for key in benign_ids[:int(ncols*nrows/2)]]
next_malignant_pix = [key for key in malignant_ids[:int(ncols*nrows/2)]]

with h5py.File('train-image.hdf5', 'r') as f:
  for i, img_key in enumerate(next_benign_pix + next_malignant_pix):
    image_data = f[img_key][()]
    image = Image.open(io.BytesIO(image_data))

    sp = plt.subplot(nrows, ncols, i + 1)
    plt.imshow(image)

plt.show()

We want to train another model on the metadata.  
The if both models say true, then the leisure is probably malignant.  
Here we select the relevant metadata for that model.

In [ ]:
COLUMNS = [
    'clin_size_long_diam_mm',
    'tbp_lv_areaMM2',
    'tbp_lv_area_perim_ratio',
    'tbp_lv_color_std_mean',
    'tbp_lv_deltaLBnorm',
    'tbp_lv_minorAxisMM',
    'tbp_lv_perimeterMM'
]

malignant_data[COLUMNS].head()

We saple the ids to get a sample data.  
We will basically use it to "test" each model before training it on huge data.

In [ ]:
import random

sample_size = 10000

benign_sample_ids = random.sample(train_benign_ids, k=sample_size)
malignant_metadata = random.sample(train_malignant_ids, k=int(sample_size/1000))

print(f'Benign sample ids: {benign_sample_ids[:5]}')
print(f'Malignant sample ids: {malignant_metadata[:5]}')

print(f'Benign sample size: {len(benign_sample_ids)}')
print(f'Malignant sample size: {len(malignant_metadata)}')

In [ ]:
from tensorflow import keras

# encoder rész
encoder = keras.models.Sequential([
    keras.layers.Reshape([96, 96, 3], input_shape=[96, 96, 3]),
    keras.layers.Conv2D(36, kernel_size=(3, 3), padding="same", activation="relu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(72, kernel_size=(3, 3), padding="same", activation="relu"),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D(144, kernel_size=(3, 3), padding="same", activation="relu"),
    keras.layers.MaxPool2D(pool_size=2)
])

encoder.summary()

In [ ]:
# decoder rész
decoder = keras.models.Sequential([
    keras.layers.Conv2DTranspose(72, kernel_size=(3, 3), strides=2, padding="same", activation="relu",
                                 input_shape=[12, 12, 144]),
    keras.layers.Conv2DTranspose(36, kernel_size=(3, 3), strides=2, padding="same", activation="relu"),
    keras.layers.Conv2DTranspose(3, kernel_size=(3, 3), strides=2, padding="same"),
    keras.layers.Reshape([96, 96, 3])
])

decoder.summary()

In [ ]:
# modell a teljes AE-re
autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.summary()

autoencoder.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
import h5py
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def hdf5_generator(hdf5_path, image_ids, batch_size, target_size=(96, 96)):
    """
    Generates batches of image data from an HDF5 file.

    Args:
        hdf5_path (str): Path to the HDF5 file.
        image_ids (list): List of image IDs to load.
        batch_size (int): Number of images per batch.
        target_size (tuple): Target size for resizing images.

    Yields:
        tuple: A batch of images and labels.
    """
    with h5py.File(hdf5_path, 'r') as f:
        num_images = len(image_ids)
        while True:
            for i in range(0, num_images, batch_size):
                batch_ids = image_ids[i: i + batch_size]
                images = []
                for image_id in batch_ids:
                    image_data = f[image_id][()]
                    image = Image.open(io.BytesIO(image_data))
                    image = image.resize(target_size)
                    image = np.array(image)
                    image = image / 255.0
                    images.append(image)

                images = np.array(images)

                yield images, images

In [ ]:
steps_per_epoch = 2000
batch_size = 128
train_len = steps_per_epoch * batch_size

print(f'Steps per epoch: {steps_per_epoch}')
print(f'Batch size: {batch_size}')
print(f'Train len: {train_len}')

train_generator = hdf5_generator('train-image.hdf5', train_benign_ids[:train_len], batch_size=batch_size)
valid_generator = hdf5_generator('train-image.hdf5', valid_benign_ids, batch_size=batch_size)

autoencoder.fit(train_generator,
                epochs = 1,
                steps_per_epoch = steps_per_epoch,
                batch_size = batch_size,
                shuffle = True)

In [ ]:
# prompt: create a function that does the following: plot an image an a prediction of the autoencoder where the image is from bengin ids. to a third column also plot the length of the differences in each pixel to a heatmap (be aware that after subtracting a vector from an other, it does not give the length, just the difference. what i want is the length)

def plot_image_and_prediction(benign_id):
  """
  Plots an image, its autoencoder prediction, and a heatmap of pixel differences.

  Args:
      benign_id (str): The ID of the benign image to process.
      autoencoder (keras.Model): The trained autoencoder model.
  """

  with h5py.File('train-image.hdf5', 'r') as f:
    image_data = f[benign_id][()]
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((96, 96))
    image = np.array(image)
    image = image / 255.0

    # Get the prediction
    prediction = autoencoder.predict(np.expand_dims(image, axis=0), verbose=0)[0]

    # Calculate the pixel difference and length
    diff = image - prediction
    diff_length = np.linalg.norm(diff, axis=2)  # Calculate length of difference in each pixel


    # Plot the image and prediction side by side
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    axs[0].imshow(image)
    axs[0].set_title('Original Image')
    axs[1].imshow(prediction)
    axs[1].set_title('Autoencoder Prediction')
    im = axs[2].imshow(diff_length, cmap='hot')
    axs[2].set_title('Difference Length (Heatmap)')
    fig.colorbar(im, ax=axs[2])

    plt.show()

# Example usage:
# Assuming you have a trained autoencoder model named 'autoencoder'
# and a list of benign IDs, e.g., 'benign_ids'
# plot_image_and_prediction(benign_ids[0], autoencoder)


In [ ]:
sample_ids = random.sample(train_benign_ids, k=3)

for image_id in sample_ids:
  plot_image_and_prediction(image_id)

In [ ]:
malignant_sample_ids = random.sample(train_malignant_ids, k=3)

for image_id in malignant_sample_ids:
  plot_image_and_prediction(image_id)

In [ ]:
# prompt: write a function that creates a 2 dimensional heatmap from an original and a predicted image

def create_heatmap(original_image, predicted_image):
  diff = original_image - predicted_image
  diff_length = np.linalg.norm(diff, axis=2)  # Calculate length of difference in each pixel
  return diff_length


In [ ]:
import numpy as np

def create_center_bitmask(heatmap_shape, center, radius):
  """Creates a bitmask for the center of a heatmap.

  Args:
    heatmap_shape (tuple): The shape of the heatmap (height, width).
    center (tuple): The coordinates of the center (cx, cy).
    radius (int): The radius of the circular center region.

  Returns:
    numpy.ndarray: A Boolean array representing the bitmask.
  """
  height, width = heatmap_shape
  cx, cy = center
  x, y = np.meshgrid(np.arange(width), np.arange(height))
  distances = np.sqrt((x - cx)**2 + (y - cy)**2)
  mask = distances <= radius
  return mask

In [ ]:
def calculate_error(image_id):
  with h5py.File('train-image.hdf5', 'r') as f:
    image_data = f[image_id][()]
    image = Image.open(io.BytesIO(image_data))
    image = image.resize((96, 96))
    image = np.array(image)

    prediction = autoencoder.predict(np.expand_dims(image, axis=0), verbose=0)[0]

    heatmap = create_heatmap(image, prediction)
    blue_channel = image[:, :, 2]
    blue_channel[blue_channel == 0] = 1000.0
    heatmap = heatmap / blue_channel

    bitmask = create_center_bitmask(heatmap.shape, (48, 48), 20)
    heatmap = heatmap * bitmask
    error = np.mean(heatmap)

    return error

In [ ]:
# prompt: fill a numpy array with the errors for bengin and an other one for malignant test images

benign_errors = []
malignant_errors = []

for image_id in test_benign_ids[:1000]:
  try:
    error = calculate_error(image_id)
    benign_errors.append(error)
  except Exception as e:
    print(f"Error processing benign image {image_id}: {e}")

for image_id in test_malignant_ids:
  try:
    error = calculate_error(image_id)
    malignant_errors.append(error)
  except Exception as e:
    print(f"Error processing malignant image {image_id}: {e}")

benign_errors_np = np.array(benign_errors)
malignant_errors_np = np.array(malignant_errors)

In [ ]:
print(f'Mean benign error: {np.mean(benign_errors_np)}')
print(f'Mean malignant error: {np.mean(malignant_errors_np)}')

In [ ]:
# prompt: create a box plot for benign_errors_np and malignant_errors_np

import matplotlib.pyplot as plt

plt.boxplot([benign_errors_np, malignant_errors_np], labels=['Benign', 'Malignant'])
plt.title('Box Plot of Reconstruction Errors')
plt.ylabel('Reconstruction Error')
plt.show()

In [ ]:
# prompt: create a box plot for benign_errors_np and malignant_errors_np and remove the very big outliers

# Remove outliers beyond 3 standard deviations
benign_errors_no_outliers = benign_errors_np[(np.abs(benign_errors_np - np.mean(benign_errors_np)) < 3 * np.std(benign_errors_np))]
malignant_errors_no_outliers = malignant_errors_np[(np.abs(malignant_errors_np - np.mean(malignant_errors_np)) < 3 * np.std(malignant_errors_np))]

# Create the box plot without outliers
plt.boxplot([benign_errors_no_outliers, malignant_errors_no_outliers], labels=['Benign', 'Malignant'])
plt.title('Box Plot of Reconstruction Errors (Outliers Removed)')
plt.ylabel('Reconstruction Error')
plt.show()

In [ ]:
# prompt: plot the confusion matrix if we predict that an image is malignant if the error is above a threshold (use benign_errors and malignant_errors)

from sklearn.metrics import confusion_matrix
import seaborn as sns

threshold = 0.1  # Adjust this threshold as needed

y_true = [0] * len(benign_errors) + [1] * len(malignant_errors)
y_pred = [(1 if error > threshold else 0) for error in benign_errors] + [(1 if error > threshold else 0) for error in malignant_errors]

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Benign', 'Malignant'], yticklabels=['Benign', 'Malignant'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
# prompt: print the f1 score for our predictions

from sklearn.metrics import f1_score

# Assuming y_true and y_pred are already defined as in your code

f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1}")